<a href="https://colab.research.google.com/github/Ueda00Ryota/Biopython-Practice/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
%cd /content/drive/My Drive/python_study/titanic

/content/drive/My Drive/python_study/titanic


In [56]:
import pandas as pd

In [57]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [58]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
#データ型の確認
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [60]:
#NULLかどうかのチェック(かけているデータの合計値)
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [61]:
test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [62]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

これらの結果からまず数値データかつnullのないものを考察していく

In [63]:
X = train[["Pclass","SibSp","Parch"]]  #使える数値data
y = train[["Survived"]]  #目的データ

K-最近傍法を用いて分析

In [64]:
from sklearn.model_selection import train_test_split
#データを分割
X_test,X_train,y_test,y_train = train_test_split(X,y,random_state = 0)

In [65]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [66]:
knn.score(X_test,y_test)

0.7005988023952096

In [67]:
X_for_submit = test[["Pclass","SibSp","Parch"]]
submit = test[["PassengerId"]] #このIDごとにsurvivedかどうかを予測したいので
submit['Survived'] = knn.predict(X_for_submit) #Survivedに予測した値が入っていく
submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [68]:
submit.to_csv('submission/submit01.csv',index = False) #indexは消しておく（三つの列にならないよう）

数値データのみでベースラインはとれたので特徴量の抽出を試みる

In [69]:
X = train[["Pclass","SibSp","Parch","Sex"]]  #性別情報を追加
y = train[["Survived"]]  #目的データ

In [70]:
X = pd.get_dummies(X,columns=["Pclass","Sex"])  #特徴的なカテゴリーデータを数値に変換
X

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,0,0,0,1,0,1
1,1,0,1,0,0,1,0
2,0,0,0,0,1,1,0
3,1,0,1,0,0,1,0
4,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...
886,0,0,0,1,0,0,1
887,0,0,1,0,0,1,0
888,1,2,0,0,1,1,0
889,0,0,1,0,0,0,1


In [71]:
X_test,X_train,y_test,y_train = train_test_split(X,y,random_state = 0)

knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

knn.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7979041916167665

In [72]:
#テストデータでnullがあったFareも抽出したい
X = train[["Pclass","SibSp","Parch","Sex","Fare"]]  #運賃を追加
y = train[["Survived"]]  #目的データ

In [74]:
X_test,X_train,y_test,y_train = train_test_split(X,y,random_state = 0)

knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

knn.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7170658682634731

In [75]:
#fareのみ数値が大きいのでそろえる→標準化
from sklearn.preprocessing import StandardScaler

scaler =  StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier()
knn.fit(X_train_scaled,y_train)

knn.score(X_test_scaled,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.7634730538922155

In [77]:
X_for_submit = test[["Pclass","SibSp","Parch","Sex","Fare"]]
submit = test[["PassengerId"]] #このIDごとにsurvivedかどうかを予測したいので

X_for_submit = pd.get_dummies(X_for_submit,columns=["Pclass","Sex"]) #特徴を数値化
X_for_submit["Fare"] = X_for_submit["Fare"].fillna(X_for_submit["Fare"].mean())  #nullにfareの平均値を入れておく
scaler =  StandardScaler()
scaler.fit(X_for_submit)   
X_for_submit_scaled = scaler.transform(X_for_submit)  #標準化

submit['Survived'] = knn.predict(X_for_submit_scaled) #Survivedに予測した値が入っていく
submit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [78]:
submit.to_csv('submission/submit02.csv',index = False) #indexは消しておく（三つの列にならないよう）

補足：
標準化では、学習データにだけfitを使うことが重要。
テストデータに対してfitさせた場合、学習データとテストデータについて異なる変換をしてしまう。

今回でいうと、scaler.fit(X_for_submit)  を消せばより良いスコアが得られる。

次にほかのモデルを試してみる

In [79]:
X = train[["Pclass","SibSp","Parch","Sex","Fare"]]  
y = train[["Survived"]]  #目的データ

X = pd.get_dummies(X,columns=["Pclass","Sex"])   #特徴抽出（数値化）

X_test,X_train,y_test,y_train = train_test_split(X,y,random_state = 0)  #データの分割

scaler =  StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)  #標準化

In [81]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train_scaled,y_train)
svc.score(X_test_scaled,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8023952095808383